In [10]:
import pandas as pd
import urllib, json, requests

# read CT accidental overdose data
deaths = pd.read_csv('data/Accidental_Drug_Related_Deaths_2012-2018.csv')

# convert date to datetime and extract year
deaths["Date"] = pd.to_datetime(deaths["Date"])
deaths["Year"] = (deaths.Date.dt.to_period('Y')).astype('str')

geoloc_match_count = 0

def fill_county_by_geoloc_match(record):
    if type(record["DeathCounty"]) == float and record["DeathCityGeo"] == record["ResidenceCityGeo"]:
        print("missing and match!")
        record["DeathCounty"] = record["ResidenceCounty"]
#         geoloc_match_count = geoloc_match_count + 1
    if type(record["ResidenceCounty"]) == float and record["DeathCityGeo"] == record["ResidenceCityGeo"]:
        print("Missing & match")
        record["ResidenceCounty"] = record["DeathCounty"] 
#         geoloc_match_count = geoloc_match_count + 1
    return record
        
deaths = deaths.apply(lambda x: fill_county_by_geoloc_match(x), axis=1)
print('break')
deaths

def find_county_by_geolocation(record, county_type):
    if county_type == "res":
        coordinates = record[-3][record[-3].find('(')+1:-1].split(', ')
    else:
        coordinates = record[-4][record[-4].find('(')+1:-1].split(', ')
    
    with urllib.request.urlopen("https://geo.fcc.gov/api/census/area?lat=" + coordinates[0] + "&lon=" + coordinates[1] + "&format=json") as url:
        data = json.loads(url.read().decode())
        county_name = data["results"][0]["county_name"]
        return county_name

def fill_county_if_nan(record):
    if type(record["ResidenceCounty"]) == float and type(record["ResidenceCityGeo"]) == str:
        print("finding res county")
        county = find_county_by_geolocation(record, 'res')
        record["ResidenceCounty"] = county.upper()
    
    if type(record["DeathCounty"]) == float:
        print("finding death county")
        county = find_county_by_geolocation(record, 'death')
        record["DeathCounty"] = county.upper()
    return record
    
deaths = deaths.apply(lambda x: fill_county_if_nan(x), axis=1)
deaths
# print(deaths.iloc[0])
# fill_county_if_nan(deaths.iloc[0])
# print(deaths.iloc[0])

missing and match!
Missing & match
missing and match!
Missing & match
Missing & match
missing and match!
Missing & match
missing and match!
missing and match!
Missing & match
missing and match!
missing and match!
missing and match!
missing and match!
Missing & match
missing and match!
missing and match!
missing and match!
Missing & match
Missing & match
missing and match!
missing and match!
Missing & match
missing and match!
missing and match!
Missing & match
missing and match!
missing and match!
missing and match!
missing and match!
missing and match!
Missing & match
missing and match!
missing and match!
Missing & match
missing and match!
Missing & match
missing and match!
missing and match!
Missing & match
Missing & match
missing and match!
missing and match!
Missing & match
missing and match!
missing and match!
missing and match!
missing and match!
missing and match!
missing and match!
Missing & match
missing and match!
missing and match!
Missing & match
missing and match!
missing a

,ID,Date,DateType,Age,Sex,Race,ResidenceCity,ResidenceCounty,ResidenceState,DeathCity,...,Morphine_NotHeroin,Hydromorphone,Other,OpiateNOS,AnyOpioid,MannerofDeath,DeathCityGeo,ResidenceCityGeo,InjuryCityGeo,Year
0,14-0273,2014-06-28,DateReported,NaN,NaN,NaN,NaN,MIDDLESEX,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,Accident,"CT\n(41.575155, -72.738288)","CT\n(41.575155, -72.738288)","CT\n(41.575155, -72.738288)",2014
1,13-0102,2013-03-21,DateofDeath,48.0,Male,Black,NORWALK,FAIRFIELD,NaN,NORWALK,...,NaN,NaN,NaN,NaN,NaN,Accident,"Norwalk, CT\n(41.11805, -73.412906)","NORWALK, CT\n(41.11805, -73.412906)","CT\n(41.575155, -72.738288)",2013
2,16-0165,2016-03-13,DateofDeath,30.0,Female,White,SANDY HOOK,FAIRFIELD,CT,DANBURY,...,NaN,NaN,NaN,NaN,Y,Accident,"Danbury, CT\n(41.393666, -73.451539)","SANDY HOOK, CT\n(41.419998, -73.282501)",NaN,2016
3,16-0208,2016-03-31,DateofDeath,23.0,Male,White,RYE,WESTCHESTER,NY,GREENWICH,...,NaN,NaN,NaN,NaN,Y,Accident,"Greenwich, CT\n(41.026526, -73.628549)",NaN,NaN,2016
4,13-0052,2013-02-13,DateofDeath,22.0,Male,"Asian, Other",FLUSHING,QUEENS,NaN,GREENWICH,...,NaN,NaN,NaN,NaN,NaN,Accident,"Greenwich, CT\n(41.026526, -73.628549)",NaN,"CT\n(41.575155, -72.738288)",2013
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
5100,15-0466,2015-09-08,DateReported,43.0,Male,White,CHESHIRE,NEW HAVEN,CT,CHESHIRE,...,NaN,NaN,NaN,NaN,NaN,Accident,"CHESHIRE, CT\n(41.498834, -72.901448)","CHESHIRE, CT\n(41.498834, -72.901448)","CT\n(41.575155, -72.738288)",2015
5101,17-0618,2017-07-22,DateReported,21.0,Male,White,MADISON,NEW HAVEN,CT,NEW HAVEN,...,NaN,NaN,NaN,NaN,NaN,Accident,"New Haven, CT\n(41.308252, -72.924161)","MADISON, CT\n(41.271447, -72.60086)","CT\n(41.575155, -72.738288)",2017
5102,18-0646,2018-08-14,DateofDeath,30.0,Male,White,LAWRENCEVILLE,TIOGA,PA,DANBURY,...,NaN,NaN,NaN,NaN,Y,Accident,"DANBURY, CT\n(41.393666, -73.451539)",NaN,"DANBURY, CT\n(41.393666, -73.451539)",2018
5103,14-0124,2014-03-16,DateofDeath,33.0,Male,White,HARTFORD,HARTFORD,NaN,WINDSOR,...,NaN,NaN,NaN,NaN,NaN,Accident,"WINDSOR, CT\n(41.852781, -72.64379)","HARTFORD, CT\n(41.765775, -72.673356)","CT\n(41.575155, -72.738288)",2014


In [11]:
death_counties = deaths.groupby(["Year","DeathCounty"])
residence_counties = deaths.groupby(["Year","ResidenceCounty"])
death_counties.get_group(('2013', 'FAIRFIELD')).set_index("DeathCounty")

death_years = deaths.groupby("Year")
death_year = death_years.get_group("2012")
death_year["ResidenceCounty"]

6         HARTFORD
8         HARTFORD
10       MIDDLESEX
17       NEW HAVEN
47         WINDHAM
           ...    
5025    NEW LONDON
5067    NEW LONDON
5082     MIDDLESEX
5086     NEW HAVEN
5094    NEW LONDON
Name: ResidenceCounty, Length: 355, dtype: object

In [12]:
counties = ('FAIRFIELD', 'HARTFORD', 'NEW HAVEN', 'NEW LONDON',
       'LITCHFIELD', 'MIDDLESEX', 'WINDHAM', 'TOLLAND')
years = ('2012', '2013', '2014', '2015', '2016', '2017', '2018')
deaths_county_and_year = []

for county in counties:
    
    residence_county = deaths[deaths["ResidenceCounty"] == county]
    death_county = deaths[deaths["DeathCounty"] == county]
    
    for year in years:
        
        deaths_county_and_year.append([
            county,
            year,
            len(residence_county[residence_county["Year"] == year]),
            len(death_county[death_county["Year"] == year])
        ])

deaths_county_and_year


[['FAIRFIELD', '2012', 51, 52],
 ['FAIRFIELD', '2013', 83, 94],
 ['FAIRFIELD', '2014', 76, 91],
 ['FAIRFIELD', '2015', 106, 103],
 ['FAIRFIELD', '2016', 159, 165],
 ['FAIRFIELD', '2017', 147, 165],
 ['FAIRFIELD', '2018', 148, 151],
 ['HARTFORD', '2012', 92, 101],
 ['HARTFORD', '2013', 119, 132],
 ['HARTFORD', '2014', 142, 166],
 ['HARTFORD', '2015', 189, 212],
 ['HARTFORD', '2016', 255, 284],
 ['HARTFORD', '2017', 284, 338],
 ['HARTFORD', '2018', 283, 339],
 ['NEW HAVEN', '2012', 90, 99],
 ['NEW HAVEN', '2013', 130, 135],
 ['NEW HAVEN', '2014', 140, 141],
 ['NEW HAVEN', '2015', 188, 211],
 ['NEW HAVEN', '2016', 233, 263],
 ['NEW HAVEN', '2017', 274, 288],
 ['NEW HAVEN', '2018', 239, 269],
 ['NEW LONDON', '2012', 43, 41],
 ['NEW LONDON', '2013', 51, 56],
 ['NEW LONDON', '2014', 49, 51],
 ['NEW LONDON', '2015', 59, 61],
 ['NEW LONDON', '2016', 85, 90],
 ['NEW LONDON', '2017', 85, 81],
 ['NEW LONDON', '2018', 89, 91],
 ['LITCHFIELD', '2012', 20, 17],
 ['LITCHFIELD', '2013', 35, 36],
 ['LI

In [13]:
deaths_by_county = pd.DataFrame(deaths_county_and_year)
deaths_by_county.columns = ('Jurisdiction', 
        'Year',
        'ResidentDeaths',
        'DeathsInCounty'
                           )
deaths_by_county["Jurisdiction"] = deaths_by_county["Jurisdiction"].str.title()
deaths_by_county.to_csv('data/deaths_by_county.csv')

In [ ]:
# deaths["DeathCounty"].unique()